In [57]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData



In [60]:
#CONNECT TO DB 
url = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(
user='postgres', passwd='Angybix6355', host='localhost', port='5432', db='country_db')

engine = create_engine(url)
connection = engine.connect()

Session = sessionmaker(bind = engine)
session = Session()
engine.table_names()

['united_nations_transformed',
 'life_expectancy',
 'world_happiness_15',
 'world_happiness_16',
 'world_happiness_17',
 'country_db',
 'free_ecos_4']

In [61]:
# GET TABLES 
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()


['united_nations_transformed',
 'life_expectancy',
 'world_happiness_15',
 'world_happiness_16',
 'world_happiness_17',
 'free_ecos_4']

In [74]:
# MAP ALIAS TO TABLES 
life = Base.classes.life_expectancy

unt = Base.classes.united_nations_transformed
wh15 = Base.classes.world_happiness_15
world_happiness_16 = Base.classes.world_happiness_16
world_happiness_17 = Base.classes.world_happiness_17



In [63]:
session = Session(engine)

TypeError: __call__() takes 1 positional argument but 2 were given

# INSPECT TABLES 

In [64]:
inspector = inspect(engine)
inspector.get_table_names()

['united_nations_transformed',
 'life_expectancy',
 'world_happiness_15',
 'world_happiness_16',
 'world_happiness_17',
 'country_db',
 'free_ecos_4']

In [65]:
columns = inspector.get_columns('life_expectancy')
for c in columns:
    print(c['name'], c["type"])

index BIGINT
country TEXT
year BIGINT
life_expectancy DOUBLE PRECISION
population DOUBLE PRECISION


In [66]:
columnss = inspector.get_columns('united_nations_transformed')
for d in columnss:
    print(d['name'], d["type"])

country TEXT
Region TEXT
pop_in_1000s BIGINT
pop_dens_per_sqkm DOUBLE PRECISION
males_per_100_fem DOUBLE PRECISION
GDP_in_mil_dol BIGINT
GDP_per_cap_in_mil DOUBLE PRECISION
unemploy_perct TEXT
pop_growth_rate_perct TEXT
live_births_per_woman TEXT


In [67]:
columnst = inspector.get_columns('world_happiness_15')
for e in columnst:
    print(e['name'], e["type"])

index BIGINT
country TEXT
happiness_rank BIGINT
happiness_score DOUBLE PRECISION


In [68]:
columnsz = inspector.get_columns('free_ecos_4')
for f in columnsz:
    print(f['name'], f["type"])

CountryID BIGINT
Country_Name TEXT
WEBNAME TEXT
Region TEXT
World_Rank DOUBLE PRECISION
Region_Rank DOUBLE PRECISION
2019_Score DOUBLE PRECISION
Property_Rights DOUBLE PRECISION
Judical_Effectiveness DOUBLE PRECISION
Government_Integrity DOUBLE PRECISION
Tax_Burden DOUBLE PRECISION
Govt_Spending DOUBLE PRECISION
Fiscal_Health DOUBLE PRECISION
Business_Freedom DOUBLE PRECISION
Labor_Freedom DOUBLE PRECISION
Monetary_Freedom DOUBLE PRECISION
Trade_Freedom DOUBLE PRECISION
Investment_Freedom DOUBLE PRECISION
Financial_Freedom DOUBLE PRECISION
Tariff_Rate DOUBLE PRECISION
Income_Tax_Rate DOUBLE PRECISION
Corporate_Tax_Rate DOUBLE PRECISION
Tax_Burden__of_GDP DOUBLE PRECISION
Govt_Expenditure_of_GDP DOUBLE PRECISION
Population_Millions DOUBLE PRECISION
GDP_Billions_PPP DOUBLE PRECISION
GDP_Growth_Rate DOUBLE PRECISION
5_Year_GDP_Growth_Rate DOUBLE PRECISION
GDP_per_Capita_PPP BIGINT
Unemployment DOUBLE PRECISION
Inflation DOUBLE PRECISION
FDI_Inflow_Millions DOUBLE PRECISION
Public_Debtof_G

# SOME COLUMN NAMES NOTES:
happiness columns 
index	country	happiness_rank	happiness_score

free columns
Country_Name
Column('World_Rank' ,Integer),
Column('Region_Rank', Integer),
Column('2019_Score', Float(10,2)),

# JOINS QUERYS BELOW :

In [69]:
# ATTEMPT THIS BASIC JOIN
data = engine.execute("SELECT * FROM free_ecos_4.Country_Name") 
data

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "free_ecos_4.country_name" does not exist
LINE 1: SELECT * FROM free_ecos_4.Country_Name
                      ^

[SQL: SELECT * FROM free_ecos_4.Country_Name]
(Background on this error at: http://sqlalche.me/e/f405)

In [70]:
# ATTEMPT BASIC JOIN 2 
join_a = session.query.filter(wh15.country == free.Country_Name).limit(10).all()

AttributeError: 'function' object has no attribute 'filter'

In [71]:
# ATTEMPT ADVANCED JOIN 3 
selection = [ wh15.country, wh15.happiness_rank, w15.happiness_score, free.Country_Name,  free.World_Rank,   free.Region_Rank  free.2019_Score]
join_B = session.query(*selection).filter(wh15.country == free.Country_Name).limit(10).all()

SyntaxError: invalid syntax (<ipython-input-71-3824b37758f5>, line 2)

In [82]:
 engine.execute( "SELECT free_ecos_4.Country_Name\
                     FROM united_nations_transformed\
                   LEFT JOIN life_expectancy\
                       ON united_nations_transformed.country = life_expectancy.country\
                   LEFT JOIN world_happiness_15\
                       ON united_nations_transformed.country = world_happiness_15.country\
                   LIMIT 10;")
    
    
    

ProgrammingError: (psycopg2.errors.UndefinedTable) missing FROM-clause entry for table "free_ecos_4"
LINE 1: SELECT free_ecos_4.Country_Name                    FROM unit...
               ^

[SQL: SELECT free_ecos_4.Country_Name                    FROM united_nations_transformed                  LEFT JOIN life_expectancy                      ON united_nations_transformed.country = life_expectancy.country                  LEFT JOIN world_happiness_15                      ON united_nations_transformed.country = world_happiness_15.country                  LIMIT 10;]
(Background on this error at: http://sqlalche.me/e/f405)

# EXAMPLES OF JOINS BELOW  :


selection = [ Measurement.station,  Station.name, Measurement.prcp, Station.latitude, Station.longitude, Station.elevation ]
    join_1_ = session.query(*selection).filter(Measurement.station == Station.station).\
    filter(Measurement.date >='2017-02-28', Measurement.date <='2017-03-05' ).\
    order_by(Measurement.prcp.desc()).limit(10).all()
    
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same sporder as the North American animal '{na_fam} {na_gen} {na_spec}'.")  